# 0. 概要

本セッションでは、名寄せを題材とした、Snowflakeの文字列処理やLLM関数を活用しながら、SnowflakeにおけるCortex LLMの活用方法を学ぶ、データエンジニアリング向けセッションである。なお、一部のパートでは皆様と手を動かすところもございます。
- <***>のところが皆様に修正して頂きたい点であります。


# 1. データの準備

この章では外部にある3種類のCSVファイルをSnowflake テーブルとして投入する

In [ ]:
use role accountadmin;
alter account set CORTEX_ENABLED_CROSS_REGION = 'AWS_US';


In [ ]:
-- データベースの作成, スキーマの準備
use schema snowretail_db.snowretail_schema;

-- ファイルを確認
ls @file;

In [ ]:
-- データの投入
-- Step1: テーブル作成
create or replace TABLE CUSTOMER_REVIEWS (
	REVIEW_ID VARCHAR(16777216),
	PRODUCT_ID VARCHAR(16777216),
	CUSTOMER_ID VARCHAR(16777216),
	RATING NUMBER(38,1),
	REVIEW_TEXT VARCHAR(16777216),
	REVIEW_DATE TIMESTAMP_NTZ(9),
	PURCHASE_CHANNEL VARCHAR(16777216),
	HELPFUL_VOTES NUMBER(38,0)
);

create or replace TABLE EC_DATA (
	TRANSACTION_ID VARCHAR(16777216),
	TRANSACTION_DATE DATE,
	PRODUCT_ID VARCHAR(16777216),
	PRODUCT_NAME VARCHAR(16777216),
	QUANTITY NUMBER(38,0),
	UNIT_PRICE NUMBER(38,0),
	TOTAL_PRICE NUMBER(38,0)
);

create or replace TABLE PRODUCT_MASTER (
	PRODUCT_ID VARCHAR(16777216),
	PRODUCT_NAME VARCHAR(16777216),
	UNIT_PRICE NUMBER(38,0)
);

create or replace TABLE RETAIL_DATA (
	TRANSACTION_ID VARCHAR(16777216),
	TRANSACTION_DATE DATE,
	PRODUCT_ID VARCHAR(16777216),
	PRODUCT_NAME VARCHAR(16777216),
	QUANTITY NUMBER(38,0),
	UNIT_PRICE NUMBER(38,0),
	TOTAL_PRICE NUMBER(38,0)
);

create or replace TABLE SNOW_RETAIL_DOCUMENTS (
	DOCUMENT_ID VARCHAR(16777216),
	TITLE VARCHAR(16777216),
	CONTENT VARCHAR(16777216),
	DOCUMENT_TYPE VARCHAR(16777216),
	DEPARTMENT VARCHAR(16777216),
	CREATED_AT TIMESTAMP_NTZ(9),
	UPDATED_AT TIMESTAMP_NTZ(9),
	VERSION NUMBER(38,1)
);

-- Step2: データロード
create or replace temp file format temp_ff
    type = csv
    skip_header = 1
; 

create or replace temp file format temp_ff_2
	TYPE=CSV
    SKIP_HEADER=1
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
; 

copy into CUSTOMER_REVIEWS
from @FILE
file_format = (format_name = temp_ff)
files = ('customer_reviews.csv');

copy into EC_DATA
from @FILE
file_format = (format_name = temp_ff)
files = ('ec_data.csv');

copy into PRODUCT_MASTER
from @FILE
file_format = (format_name = temp_ff)
files = ('product_master.csv');

copy into RETAIL_DATA
from @FILE
file_format = (format_name = temp_ff)
files = ('retail_data.csv');

copy into SNOW_RETAIL_DOCUMENTS
from @FILE
file_format = (format_name = temp_ff_2)
files = ('snow_retail_documents.csv');

# 2. データのチェック

この章では1章で入れたテーブルデータの中身を確認する

商品名のパターン
- 商品名には大文字／小文字変換、半角カタカナ変換、全角変換が施されている
- メーカーと商品には空白の有無、ハイフンの除去、接尾語の追加などがされている
- 商品名の末尾には、[半額セール]、[期間限定]、[閉店セール] などのタグが入っている
- 小売やECのデータにはメーカー名がない製品が存在している

In [ ]:
-- MASTER DATA
select * from product_master;

In [ ]:
-- RETAIL DATA
select * from retail_data limit 100;

In [ ]:
-- EC DATA
select * from ec_data limit 100;

何も加工処理せずに、プロダクト名だけで突合しようとしてもできない

In [ ]:
select
    count(*) as record_cnt
from product_master as a
inner join retail_data as b
on contains(a.product_name,b.product_name)
limit 100;

In [ ]:
select
    count(*) as record_cnt
from product_master as a
inner join ec_data as b
on contains(a.product_name,b.product_name)
limit 100;

## 3. データの加工処理（名寄せ）

## 不要なタグを削除

処理内容:
1.	正規表現 r'[\[\【\（\＜\［].*?[\]\】\）\＞\］]' は、角括弧、全角括弧、その他囲い文字で囲まれた部分を検出して削除する。
2.	さらに「店頭」や「ネット」といった特定の語句も除去し、余分な表記が残らないようにしてする。
3.	複数の空白文字（スペース、タブ、改行）をすべて取り除く。

***皆さんがSQLを書くところです***

In [ ]:
WITH normalized_ec_data AS(
    SELECT
      PRODUCT_NAME AS ORIGINAL_NAME,
      lower(
        <***>(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      ) AS NORMALIZED_PRODUCT_NAME
    FROM ec_data
)

select 
    NORMALIZED_PRODUCT_NAME
    ,count(*) as record_cnt
from normalized_ec_data
group by all
order by NORMALIZED_PRODUCT_NAME

処理内容
- 全角半角文字を統一する

***皆さんがSQLを書くところです***

In [ ]:
WITH normalized_ec_data AS(
    SELECT
      PRODUCT_NAME AS ORIGINAL_NAME,
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      AS NORMALIZED_PRODUCT_NAME
    FROM ec_data
)

select 
    <***>(NORMALIZED_PRODUCT_NAME, 'unicode-ci') as NORMALIZED_PRODUCT_NAME
    ,count(*) as record_cnt
from normalized_ec_data
group by all
order by NORMALIZED_PRODUCT_NAME

## 正規表現を適用して突合

In [ ]:
WITH normalized_retail_data AS(
    SELECT
      *,
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      AS NORMALIZED_PRODUCT_NAME
    FROM retail_data
),

normalized_product_master AS(
    SELECT
      product_id as product_id_master,
      PRODUCT_NAME AS ORIGINAL_NAME_master,
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      AS NORMALIZED_PRODUCT_NAME_master
    FROM product_master
),

retail_joined_master_product AS(
    select 
        b.* exclude(NORMALIZED_PRODUCT_NAME_master),
        a.* exclude(product_id, NORMALIZED_PRODUCT_NAME),
        case when a.product_id = b.product_id_master then 1 else 0 end correct_flg
    from normalized_retail_data as a
    left outer join normalized_product_master as b
    on contains(collate(b.NORMALIZED_PRODUCT_NAME_master, 'unicode-ci'), collate(a.NORMALIZED_PRODUCT_NAME, 'unicode-ci'))
) 

select
    *
-- count(*)
from retail_joined_master_product
-- where product_id_master is not null
limit 100;

In [ ]:
WITH normalized_ec_data AS(
    SELECT
      *,
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
     AS NORMALIZED_PRODUCT_NAME
    FROM ec_data
),

normalized_product_master AS(
    SELECT
      product_id as product_id_master,
      PRODUCT_NAME AS ORIGINAL_NAME_master,
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
     AS NORMALIZED_PRODUCT_NAME_master
    FROM product_master
),

ec_joined_master_product AS(
    select 
        b.* exclude(NORMALIZED_PRODUCT_NAME_master),
        a.* exclude(product_id, NORMALIZED_PRODUCT_NAME),
        case when a.product_id = b.product_id_master then 1 else 0 end correct_flg
    from normalized_ec_data as a
    left outer join normalized_product_master as b
    on contains(collate(b.NORMALIZED_PRODUCT_NAME_master, 'unicode-ci'), collate(a.NORMALIZED_PRODUCT_NAME, 'unicode-ci'))
) 

select
    *
-- count(*)
from ec_joined_master_product
-- where product_id_master is not null
limit 100;

## 3. ベクトル検索の利用

この章では商品名をベクトル化した後に、類似度計算からマスターデータと購買履歴のデータを突合する

### 小売、ECの商品名に対してEmbeddingを適用し、ベクトル化を実施
***皆さんがSQLを書くところです***

In [ ]:
-- MASTER DATA
create or replace table product_master_embed as 
select 
    *, 
    snowflake.cortex.<***>('multilingual-e5-large', product_name) as product_name_embed 
from product_master;

In [ ]:
-- RETAIL DATA
create or replace table retail_data_embed as 
select 
    *, 
    snowflake.cortex.embed_text_1024('multilingual-e5-large', product_name) as product_name_embed 
from retail_data;

In [ ]:
-- EC DATA
create or replace table ec_data_embed as 
select 
    *, 
    snowflake.cortex.embed_text_1024('multilingual-e5-large', product_name) as product_name_embed 
from ec_data;

In [ ]:
-- 確認する
select * from retail_data_embed limit 10;

In [ ]:
-- マスタと小売のデータを結合し、コサイン類度の高い（0.9以上）レコードを抽出
with match_retail_product_master as (
    select 
        a.* exclude(product_name_embed)
        ,b.* exclude(product_id, product_name_embed)
        ,vector_cosine_similarity(a.product_name_embed, b.product_name_embed) as similarity
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
    from product_master_embed a, retail_data_embed b
    where vector_cosine_similarity(a.product_name_embed, b.product_name_embed) > 0.9
)

select * from match_retail_product_master limit 100;

In [ ]:
-- マスタとECのデータを結合し、コサイン類度の高い（0.9以上）レコードを抽出
with match_ec_product_master as (
    select 
        a.* exclude(product_name_embed)
        ,b.* exclude(product_id, product_name_embed)
        ,vector_cosine_similarity(a.product_name_embed, b.product_name_embed) as similarity
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
    from product_master_embed a, ec_data_embed b
    where vector_cosine_similarity(a.product_name_embed, b.product_name_embed) > 0.9
)

select count(distinct transaction_id) from match_ec_product_master limit 1000;

### ベクトル検索精度を向上

Embeddingを適用する前に生データから不要なデータを抽出するだけでも精度を向上させることができる

In [ ]:
create or replace table normalized_ec_data_embed as 
with normalized_ec_data as(
    SELECT
      *,
      upper(
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      ) AS normalized_product_name
    FROM ec_data
)

select 
    *, 
    snowflake.cortex.embed_text_1024('multilingual-e5-large', collate(normalized_product_name, 'unicode-ci')) as normalized_product_name_embed 
from normalized_ec_data;

In [ ]:
create or replace table normalized_retail_data_embed as 
with normalized_retail_data as(
    SELECT
      *,
      upper(
        regexp_replace(
          regexp_replace(
            regexp_replace(PRODUCT_NAME, '[\[\【\（\＜\［].*?[\]\】\）\＞\］]', ''),
            '(店頭|ネット)',
            ''
          ),
          '\\s+',
          ''
        )
      ) AS normalized_product_name
    FROM retail_data
)

select 
    *, 
    snowflake.cortex.embed_text_1024('multilingual-e5-large', collate(normalized_product_name, 'unicode-ci')) as normalized_product_name_embed 
from normalized_retail_data;

In [ ]:
-- 作成し直したテーブルを利用し、コサイン類似度が0.9以上でどれくらい紐づいたかを確認する
with match_ec_product_master as (
    select 
        a.* exclude(product_name_embed)
        ,b.* exclude(product_id, normalized_product_name_embed)
        ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
    from product_master_embed a, normalized_ec_data_embed b
    where vector_cosine_similarity(product_name_embed, normalized_product_name_embed) > 0.9 

)

-- select * from match_ec_product_master limit 1000;
select count(distinct transaction_id) from match_ec_product_master limit 1000;

- データをクレンジングすることでも精度向上が見込める
- Embedding用モデル自体を変更することでも精度向上が見込める。Snowflakeではvoyageやsnowflake arcticなどがある。


In [ ]:
-- 紐づいていないデータの確認
with match_ec_product_master as (
    select 
        a.* exclude(product_name_embed)
        ,b.* exclude(product_id, normalized_product_name_embed)
        ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
    from product_master_embed a, normalized_ec_data_embed b
    where vector_cosine_similarity(product_name_embed, normalized_product_name_embed) < 0.9 

)

select 
    *
from match_ec_product_master 
where correct_flg = 1
limit 100;

## 4. Cortex LLMの活用方法

ベクトル検索で紐づかないデータに対して、LLMを活用していく

## LLMでメーカーと商品名を分離

ここでは、Snowflake Cortexの関数の一つである**Complete**関数を利用し、Anthropic社の**Claude3.5**モデルでメーカーと商品名を分離する。また、出力形式を統一するために、**Cortex Complete Structure Output**の機能を利用し、JSON形式で結果を出力させる。

***皆さんがSQLを書くところです***

In [ ]:
create or replace table product_master_applied_llm as
with normalized_product_master as(
    select
        *,
        snowflake.cortex.<***>('claude-3-5-sonnet', [
            {
                'role': 'user',
                'content': concat('入力値からメーカーと商品名を分離してください。なお、必ずしもメーカー名があるわけではなく、メーカー名がない場合は空欄で返してください。入力値は',product_name,'です')
            }
            ],
            {
            'temperature': 0.7,
            'max_tokens': 8000,
            'response_format': {
                'type': 'json',
                'schema': {
                    'type': 'object',
                    'properties': {
                        'sub_maker_name': {
                            'type': 'string',
                            'description': 'メーカーの名前'
                        },
                        'sub_product_name': {
                            'type': 'string',
                            'description': '商品の名前'
                        }
                    }
                }
            }
        }) as result_json
    from product_master
),

normalized_product_master_2 as (
    select 
        *,
        result_json:structured_output[0].raw_message.sub_maker_name::varchar as sub_maker_name,
        result_json:structured_output[0].raw_message.sub_product_name::varchar as sub_product_name
    from normalized_product_master
)

select 
    *
from normalized_product_master_2;

In [ ]:
--結果の確認
select * from product_master_applied_llm limit 100

In [ ]:
-- 分離した結果（商品名）をベクトル化する
create or replace table product_master_llm_embed as 
select 
    *, 
    snowflake.cortex.embed_text_1024('multilingual-e5-large', sub_product_name) as product_name_embed 
from product_master_applied_llm;

In [ ]:
-- 元のベクトル情報を元に、コサイン類似度を計算する
with match_ec_product_master as (
    select 
        a.*  exclude(product_name_embed)
        ,b.* exclude(product_id)
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
        ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity_before
    from product_master_embed a, normalized_ec_data_embed b
    where vector_cosine_similarity(product_name_embed, normalized_product_name_embed) < 0.9 

), 

-- 精度の低いレコードのみを抽出
match_ec_product_master_2 as (
    select 
        *
    from match_ec_product_master 
    where correct_flg = 1
),

-- 新規に作成したベクトル情報を使用し、再度、コサイン類似度を最計算する
match_ec_product_master_3 as (
    select 
        a.* exclude(product_name_embed)
        ,b.* exclude(product_id, normalized_product_name_embed)
        ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity_after
    from product_master_llm_embed a, match_ec_product_master_2 b
    where vector_cosine_similarity(product_name_embed, normalized_product_name_embed) > 0.9 

)

select * from match_ec_product_master_3;

## データ加工におけるLLMの利用シーン

今回のように正規表現では手間のかかる処理をLLMに実行させるというのは非常にわかりやすい例である。
また他のユースケースとしては
- データ分類やメタデータの付与（例. 家電、テレビ、衣類など抽象列を作成する）
- 欠損値の補完
- 表記揺れの統一（住所などの異なるフォーマットの統一）

などLLMを利用することで、従来では難しかった処理を容易に解決できるかもしれない

## LLM as a Judgement

LLMの別の使い方として、従来は人間がやっていた出力結果に対する確認処理を、LLMに代行させるというLLM as a Judgementにも活用することができる。今回の例では相対的にコサイン類度が低い結果に対してLLMに判定させる処理を実施する

In [ ]:
-- 元のベクトル情報を元に、コサイン類似度を計算する
with match_ec_product_master as (
    select 
        a.product_id
        ,a.product_name as product_name_master
        ,b.* exclude(product_id)
        ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
        ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity_before
    from product_master_embed a, normalized_ec_data_embed b
    where vector_cosine_similarity(product_name_embed, normalized_product_name_embed) < 0.9 

), 

-- 精度の低いレコードのみを抽出
match_ec_product_master_2 as (
    select 
        *
    from match_ec_product_master 
    where correct_flg = 1
    limit 20
),

match_ec_product_master_3 as (
    select
        *,
        snowflake.cortex.complete('claude-3-5-sonnet', [
            {
                'role': 'user',
                'content': concat('あなたは、二つの異なる名前の商品が入力された情報を元に同じかどうかを判定するスペシャリストです。
                次のルールに従って結果を出力してください。
                1. 類似度のスコアを0から1でNumericな値で返すこと。 0が異なる製品で、1が同一製品を指す。
                2. その選択した理由も返してください。
                比較して欲しい情報は',product_name_master,'と', product_name, 'です')
            }
            ],
            {
            'temperature': 0.7,
            'max_tokens': 8000,
            'response_format': {
                'type': 'json',
                'schema': {
                    'type': 'object',
                    'properties': {
                        'score': {
                            'type': 'number',
                            'description': 'メーカーの名前'
                        },
                        'reason': {
                            'type': 'string',
                            'description': 'スコアの理由'
                        }
                    }
                }
            }
        }) as result_json
    from match_ec_product_master_2
), 

match_ec_product_master_4 as (
    select 
        *,
        result_json:structured_output[0].raw_message.score::float as llm_score,
        result_json:structured_output[0].raw_message.reason::varchar as llm_reason
    from match_ec_product_master_3
)

select 
    *
from match_ec_product_master_4;

## 5.最終結果の作成

In [ ]:
create or replace table ec_data_with_product_master as 

    with combined_product_master as (
        select a.*, b.product_name_embed as product_name_llm_embed from product_master_embed as a
        inner join product_master_llm_embed as b
        on a.product_id = b.product_id
    ),
    
    match_ec_product_master as (
        select 
            a.product_id as product_id_master
            ,a.product_name as product_name_master
            ,a.unit_price as unit_price_master 
            ,b.* exclude(product_id, normalized_product_name, normalized_product_name_embed)
            ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity_before
            ,vector_cosine_similarity(a.product_name_llm_embed, b.normalized_product_name_embed) as similarity_after
            ,greatest(similarity_before, similarity_after) as similarity -- beforeとafterを比較して高い方の値を出す
            ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
        from combined_product_master a, normalized_ec_data_embed b
    
    ), 
    
    match_ec_product_master_2 as (
        select 
            * exclude(similarity_before, similarity_after, correct_flg)
        from match_ec_product_master 
        where similarity > 0.9
    )
    
    
    select * from match_ec_product_master_2


In [ ]:
create or replace table retail_data_with_product_master as 

    with combined_product_master as (
        select a.*, b.product_name_embed as product_name_llm_embed from product_master_embed as a
        inner join product_master_llm_embed as b
        on a.product_id = b.product_id
    ),
    
    match_retail_product_master as (
        select 
            a.product_id as product_id_master
            ,a.product_name as product_name_master
            ,a.unit_price as unit_price_master 
            ,b.* exclude(product_id, normalized_product_name, normalized_product_name_embed)
            ,vector_cosine_similarity(a.product_name_embed, b.normalized_product_name_embed) as similarity_before
            ,vector_cosine_similarity(a.product_name_llm_embed, b.normalized_product_name_embed) as similarity_after
            ,greatest(similarity_before, similarity_after) as similarity -- beforeとafterを比較して高い方の値を出す
            ,case when a.product_id = b.product_id then 1 else 0 end correct_flg
        from combined_product_master a, normalized_retail_data_embed b
    
    ), 
    
    match_retail_product_master_2 as (
        select 
            * exclude(similarity_before, similarity_after, correct_flg)
        from match_retail_product_master 
        where similarity > 0.9
    )
    
    
    select * from match_retail_product_master_2


In [ ]:
select * from retail_data_with_product_master limit 100;

## (オプショナル） Cortex Searchの利用

上記の一連処理を容易にアプリケーション化することが可能なサービスである

In [ ]:
-- ウェアハウスの作成
create or replace warehouse cortex_search_wh with WAREHOUSE_SIZE='X-SMALL';

-- パラメータをオン
alter table product_master set change_tracking = true;

-- Cortex Search オブジェクトを作成する
create or replace cortex search service product_master_service
  on product_name
  attributes product_name
  warehouse = cortex_search_wh
  target_lag = '1 day'
  embedding_model = 'voyage-multilingual-2'
  as (
    select * from product_master
  );
  

In [ ]:
-- 作成したCortex Search Serviceを確認する
SELECT
  *
FROM
  TABLE (
    cortex_search_data_scan(
      SERVICE_NAME => 'product_master_service'
    )
  );

In [ ]:
-- SQLを用いて、SEARCH_PREVIEWを利用し、任意のクエリを投げてみる
SELECT
  snowflake.cortex.search_preview (
      'product_master_service',
      '{
          "query": "一番搾りビール",
          "columns": ["product_name"],
          "limit": 2
      }'
  );

In [ ]:
# Pythonを用いて、Cortex Search Serviceにアクセスする。
# なお、実行する前に右上のパッケージで[snowflake, snowflake-ml-python, snowflake-snowpark-python]をインポートする
from snowflake.core import Root
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
import json

session = get_active_session()
root = Root(session)

product_master_search_service = (root
  .databases["snowretail_db"]
  .schemas["snowretail_schema"]
  .cortex_search_services["product_master_service"]
)

resp = product_master_search_service.search(
  query="一番搾りビール",
  columns=["product_name"],
  limit=2
)

# 一度、JSON文字列を辞書型に変換する
resp_dict = json.loads(resp.to_json())

# 改めてjson.dumpsで日本語を正しく表示する
print(json.dumps(resp_dict, ensure_ascii=False, indent=2))

Cortex Searchは変数による代入が対応しておらず、リテラルのみに対応している。つまり、テーブルデータを一括で突合するというのができないというのが現状である。